
# GP_ PREDICTING THE STOCK MARKET



In this project, we'll be working with data from the S&P500 Index. The S&P500 is a stock market index. 

We'll be working with a csv file containing index prices. Each row in the file contains a daily record of the price of the S&P500 Index from 1950 to 2015. The dataset is stored in sphist.csv.


The columns of the dataset are:


* `Date` -- The date of the record.

* `Open` -- The opening price of the day (when trading starts).

* `High` -- The highest trade price during the day.

* `Low` -- The lowest trade price during the day.

* `Close` -- The closing price for the day (when trading is finished).

* `Volume` -- The number of shares traded.

* `Adj Close` -- The daily closing price, adjusted retroactively to include any corporate actions.


You'll be using this dataset to develop a predictive model. You'll train the model with data from 1950-2012, and try to make predictions from 2013-2015.


In this version (V2), we will use rolling function for calculating indicators in section 2. It makes the calculation of indicators much faster.


## 1. INTRODUCTION TO DATA

In [66]:

import pandas as pd
from datetime import datetime
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold, cross_val_score


df = pd.read_csv("sphist.csv")

# for an overview
# df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       16590 non-null  object 
 1   Open       16590 non-null  float64
 2   High       16590 non-null  float64
 3   Low        16590 non-null  float64
 4   Close      16590 non-null  float64
 5   Volume     16590 non-null  float64
 6   Adj Close  16590 non-null  float64
dtypes: float64(6), object(1)
memory usage: 907.4+ KB


In [67]:

# convert values of this column to datetime
df["Date"] = pd.to_datetime(df["Date"])


# sorting for next steps. Reset index, and drop previous index, for avoiding confusion later
df_sort = df.sort_values(by = ["Date"], ascending = True).reset_index(drop = True)


df_sort.head()


,Date,Open,High,Low,Close,Volume,Adj Close
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
3,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
4,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08



## 2. GENERATING INDICATORS.


Datasets taken from the stock market need to be handled differently than datasets from other sectors when it comes time to make predictions. In a normal machine learning exercise, we treat each row as independent. Stock market data is sequential, **and each observation comes a day after the previous observation. Thus, the observations are not all independent, and you can't treat them as such.**

This means you have to be extra careful to not inject "future" knowledge into past rows when you do training and prediction. Injecting future knowledge will make our model look good when you're training and testing it, but will make it fail in the real world. This is how many algorithmic traders lose money.

The time series nature of the data means that can generate indicators to make our model more accurate. For instance, you can create a new column that contains the average price of the last 10 trades for each row. This will incorporate information from multiple prior rows into one, and will make predictions much more accurate.

When you do this, you have to be careful **not to use the current row in the values you average.** You want to teach the model how to predict the current price from historical prices. If you include the current price in the prices you average, it will be equivalent to handing the answers to the model upfront, and will make it impossible to use in the "real world", where you don't know the price upfront.


Here are some indicators that are interesting to generate for each row:


* The average price from the past 5 days.

* The average price for the past 30 days.

* The average price for the past 365 days.

* The ratio between the average price for the past 5 days, and the average price for the past 365 days.

* The standard deviation of the price over the past 5 days.

* The standard deviation of the price over the past 365 days.

* The ratio between the standard deviation for the past 5 days, and the standard deviation for the past 365 days.


**"Days" means "trading days"** -- so if you're computing the average of the past 5 days, it should be the 5 most recent dates before the current one. **Assume that "price" means the Close**


In [68]:
# we uses rolling function to get the mean of the 5 previous values of column "Close",
# then we should move this mean value one position down respect the index with shift function
# when there aren't enough values, rolling function returns nan
df_sort["avg_5"] = df_sort["Close"].rolling(5).mean().shift(1)
df_sort["avg_30"] = df_sort["Close"].rolling(30).mean().shift(1)
df_sort["avg_365"] = df_sort["Close"].rolling(365).mean().shift(1)

# same procedure for std calcultions
df_sort["std_5"] = df_sort["Close"].rolling(5).std().shift(1)
df_sort["std_365"] = df_sort["Close"].rolling(365).std().shift(1)

# for ratios calculation
df_sort["rat_avg"] = df_sort["avg_5"] / df_sort["avg_365"]
df_sort["rat_std"] = df_sort["std_5"] / df_sort["std_365"]


# df_sort.head(10)
df_sort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       16590 non-null  datetime64[ns]
 1   Open       16590 non-null  float64       
 2   High       16590 non-null  float64       
 3   Low        16590 non-null  float64       
 4   Close      16590 non-null  float64       
 5   Volume     16590 non-null  float64       
 6   Adj Close  16590 non-null  float64       
 7   avg_5      16585 non-null  float64       
 8   avg_30     16560 non-null  float64       
 9   avg_365    16225 non-null  float64       
 10  std_5      16585 non-null  float64       
 11  std_365    16225 non-null  float64       
 12  rat_avg    16225 non-null  float64       
 13  rat_std    16225 non-null  float64       
dtypes: datetime64[ns](1), float64(13)
memory usage: 1.8 MB


In [69]:
# for testing that results match guided proyect

df_sort[df_sort["Date"] == datetime(year = 1951, month = 1, day = 3)]

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30,avg_365,std_5,std_365,rat_avg,rat_std
250,1951-01-03,20.690001,20.690001,20.690001,20.690001,3370000.0,20.690001,20.36,19.815,NaN,0.304385,NaN,NaN,NaN



## 3. DROP NaN VALUES AND SPLITTING UP DATA

We will drop all rows with Nan values of previous new columns calcultations. Due to we needed more than 365 trading days for calculate `avg_365` column, this means than we will lose surely more than a year of data.


In [70]:
df_sort = df_sort.dropna(axis = 0)


df_sort.head() # we will see what is now the first date of our dateframe (we have lost the rest)


,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30,avg_365,std_5,std_365,rat_avg,rat_std
365,1951-06-19,22.020000,22.020000,22.020000,22.020000,1100000.0,22.020000,21.800,21.703333,19.447726,0.256223,1.790253,1.120954,0.143121
366,1951-06-20,21.910000,21.910000,21.910000,21.910000,1120000.0,21.910000,21.900,21.683000,19.462411,0.213659,1.789307,1.125246,0.119409
367,1951-06-21,21.780001,21.780001,21.780001,21.780001,1100000.0,21.780001,21.972,21.659667,19.476274,0.092574,1.788613,1.128142,0.051758
368,1951-06-22,21.549999,21.549999,21.549999,21.549999,1340000.0,21.549999,21.960,21.631000,19.489562,0.115108,1.787659,1.126757,0.064390
369,1951-06-25,21.290001,21.290001,21.290001,21.290001,2440000.0,21.290001,21.862,21.599000,19.502082,0.204132,1.786038,1.121008,0.114293


In [71]:
# Here we have calculated the size of each portion
train = df_sort[df_sort["Date"] < datetime(year = 2013, month = 1, day = 1)]
test = df_sort[df_sort["Date"] >= datetime(year = 2013, month = 1, day = 1)]

print()
print("train: {} values ==> {}%".format(len(train), round((len(train) / len(df_sort)) * 100, 2)))
print()
print("test: {} values =====>  {}%".format(len(test), round((len(test) / len(df_sort)) * 100, 2)))


train: 15486 values ==> 95.45%

test: 739 values =====>  4.55%



## 4. SELECTING INDICATORS FOR OUR MODEL


In [72]:

# It takes all new columns added previously, plus "Close" and calculates corr. coeficients
train_cols = [col for col in df_sort.columns if (col.startswith("avg") or col.startswith("std")
                                                or col.startswith("rat"))]
train_cols.append("Close")

df_sort[train_cols].corr()

,avg_5,avg_30,avg_365,std_5,std_365,rat_avg,rat_std,Close
avg_5,1.000000,0.999472,0.989235,0.723713,0.816110,0.047077,0.088080,0.999793
avg_30,0.999472,1.000000,0.990682,0.734546,0.816383,0.037703,0.096957,0.999189
avg_365,0.989235,0.990682,1.000000,0.765109,0.842691,-0.057614,0.113230,0.988870
std_5,0.723713,0.734546,0.765109,1.000000,0.721412,-0.174687,0.413709,0.722414
std_365,0.816110,0.816383,0.842691,0.721412,1.000000,-0.105164,-0.035165,0.816103
rat_avg,0.047077,0.037703,-0.057614,-0.174687,-0.105164,1.000000,-0.265775,0.047782
rat_std,0.088080,0.096957,0.113230,0.413709,-0.035165,-0.265775,1.000000,0.087018
Close,0.999793,0.999189,0.988870,0.722414,0.816103,0.047782,0.087018,1.000000



According to this matrix of correlation values, we will chose those indicators with a higher correlation value respect `Close` column. Thus we will descart `rat_avg` and `rat_std`.



## 5. CALCULATING BIAS AND VARIABILITY OF OUR MODEL


In [73]:
def train_and_test(df, cols, k):  
    
    
    lr = LinearRegression()
    
    # to calculate in sample error as a reference
    if k == 0:
        
        sample = df
        
        lr.fit(sample[cols], sample["Close"])
        predictions = lr.predict(sample[cols])
        mae = mean_absolute_error(sample["Close"], predictions)
        var_pred = np.var(predictions) # variance of prediction values
        
        return mae, var_pred
    
    # train and test validation
    if k == 1:
        
        train = df[df["Date"] < datetime(year = 2013, month = 1, day = 1)]
        test = df[df["Date"] >= datetime(year = 2013, month = 1, day = 1)]

        lr.fit(train[cols], train["Close"])
        predictions = lr.predict(test[cols])
        mae = mean_absolute_error(test["Close"], predictions)
        var_pred = np.var(predictions) # variance of prediction values
#         mse = mean_squared_error(test["Close"], predictions)
#         rmse = mse ** 0.5
#         return rmse, var_pred
        return mae, var_pred


    # for k values higher than 1
    else:
        
        # instance to KFold class, with k as parameter
        kf = KFold(n_splits = k, shuffle=True, random_state=1) # especific rs for getting same results
        
        # neg mean absolute errors list
        maes = cross_val_score(lr, df[cols], 
                               df["Close"], scoring = "neg_mean_absolute_error", cv =kf)
        
        # convert to positive
        maes = [abs(n) for n in maes]

        avg_maes = np.mean(maes)
        
        std_maes = np.std(maes)
        
        
        # dupla
        return avg_maes, std_maes

    
# train_cols = [col for col in df_sort.columns if (col.startswith("avg") or col.startswith("std"))]
# reset train_cols variable and only takes avg indicators
train_cols = [col for col in df_sort.columns if (col.startswith("avg"))]
    
for k in range(10):
    
    
    results = train_and_test(df_sort, train_cols, k)
    
    if k == 0:
        
        print("MAE = {:.2f} and VAR = {:.2f}, for in sample predictions".format(results[0], results[1]))
    
    elif k == 1:
        
        print("MAE = {:.2f} and VAR = {:.2f} for predictions of test and train validation."
              .format(results[0], results[1]))
        
    elif k == 2:
        
        print("AvgMAE = {:.2f} and StdMAE = {:.2f}, for holdout validation."
              .format(results[0], results[1]))
        
    else:
        
        print("AvgMAE = {:.2f} and StdMAE = {:.2f}, for k({}) fold validation."
              .format(results[0], results[1], k))

MAE = 5.48 and VAR = 309881.73, for in sample predictions
MAE = 16.14 and VAR = 37385.37 for predictions of test and train validation.
AvgMAE = 5.49 and StdMAE = 0.04, for holdout validation.
AvgMAE = 5.49 and StdMAE = 0.06, for k(3) fold validation.
AvgMAE = 5.49 and StdMAE = 0.07, for k(4) fold validation.
AvgMAE = 5.48 and StdMAE = 0.13, for k(5) fold validation.
AvgMAE = 5.48 and StdMAE = 0.14, for k(6) fold validation.
AvgMAE = 5.49 and StdMAE = 0.17, for k(7) fold validation.
AvgMAE = 5.48 and StdMAE = 0.21, for k(8) fold validation.
AvgMAE = 5.48 and StdMAE = 0.23, for k(9) fold validation.



After running this test and trail validation function for our model with different kinds of validations, we realized that the calculated prediction errors with k cross fold validation are almost the same that the one in sample error. 
It's strange because they should be higher, k cross validation is a test in which at each iteration we treat a part of the observations as out of sample, but at the end of the process, we really use all data for training the model.

This results might be caused because the depencende between observations, and the use of observation indicators based really in previous observations.

However the test and train validation, in this case, migh give us a better reference for the real bias and variance of predictions, because in this validation we treat a part of the observation as out of the sample and don't use this part for training the model at all.

It has to be said than they seem a great results in terms of accuracy and variability of our predictions if we take a look to the statistic values of Close prices in the predicted period (cell bellow).

A MAE (mean absolute error) of 16.14 in the predictions for this period, represents minus than 1% if we consider the mean of Close prices values for this period.


In [74]:
df_sort[df_sort["Date"] >= datetime(year = 2013, month = 1, day = 1)].loc[:, "Close"].describe()

count     739.000000
mean     1874.890338
std       192.389058
min      1457.150024
25%      1697.450012
50%      1920.239990
75%      2052.165039
max      2130.820068
Name: Close, dtype: float64

## 6. PREDICTING FUTURE STOCK PRICES ONE DAY AHEAD


If we would like to use this model, in the real world, for predicting stock prices at future dates, we didn't have the indicators values for doing predictions on these future observations. 

At most we will have the indicators for doing one day ahead predictions. 


In [75]:
# this function predicts price "Close" one day ahead from an indicated date
# using our linear regresion model, and all previus observations to this date as train set

def one_day_ahead(date, df, cols):
    
    
    try:
        # index position of date in df
        index = df[df["Date"] == date].index[0]
        
    except:
        
        print("This date not doen't exits in observations")
        
        return
        
    lr = LinearRegression()
         
    X = df.loc[:index] # all observation previous date
    Y = df.loc[index]
        
    lr.fit(X[cols], X["Close"])
    prediction = lr.predict([Y[cols]]) # OJO it must be in brakets, need to be array 2D
        
    return prediction[0]
    
# only takes average indicators
train_cols = [col for col in df_sort.columns if (col.startswith("avg"))]

# we can test with any date, if this exits in df_sort
one_day_ahead(datetime(year = 2013, month = 1, day = 7), df_sort, train_cols) 

1445.2562423251716

In [76]:
# in this code bellow we'll apply one_day_ahead() function to observations from 2013/01/01 
# to the end of our df, and store predictions in column Pred_price of a new dataframe created for this

df_test = df_sort[df_sort["Date"] >= datetime(year = 2013, month = 1, day = 1)].copy()

df_test["Pred_price"] = df_test.apply(lambda row: one_day_ahead(row.loc["Date"], 
                                                                df_sort, train_cols), axis = 1)


# for a better view, remove columns we don't need here
df_test = df_test.drop(df_test.columns[5:14], axis = 1)

df_test.head()

,Date,Open,High,Low,Close,Volume,Adj Close,avg_5,avg_30,avg_365,std_5,std_365,rat_avg,rat_std,Pred_price
15851,2013-01-02,1426.189941,1462.430054,1426.189941,1462.420044,4.202600e+09,1462.420044,1418.641992,1414.258667,1327.534055,9.820801,90.463948,1.068629,0.108560,1419.209222
15852,2013-01-03,1462.420044,1465.469971,1455.530029,1459.369995,3.829730e+09,1459.369995,1425.793994,1417.676668,1327.908247,22.261321,90.738976,1.073714,0.245334,1426.667952
15853,2013-01-04,1459.369995,1467.939941,1458.989990,1466.469971,3.424290e+09,1466.469971,1433.702002,1420.092668,1328.224877,26.274326,90.995857,1.079412,0.288742,1435.020145
15854,2013-01-07,1466.469971,1466.469971,1456.619995,1461.890015,3.304970e+09,1461.890015,1443.376001,1422.714665,1328.557617,27.945242,91.279049,1.086423,0.306152,1445.256242
15855,2013-01-08,1461.890015,1461.890015,1451.640015,1457.150024,3.601600e+09,1457.150024,1455.267993,1425.076664,1328.898603,16.453319,91.544368,1.095093,0.179731,1457.897557


In [77]:
mae = mean_absolute_error(df_test["Close"], df_test["Pred_price"])
var = np.var(df_test["Pred_price"])

print(mae)
print(var)


16.08815132078325
37444.44664786434



Comparing these results above with those we got in our previous train and test validation, we can see that we get almost the same values in accuracy and variability for our one_day_ahead model. There is a slighty improve in median absolute error and a little more of variability in our predictions, but not great deal.



## 7. PREDICTING STOCK PRICES FURTHER AWAY THAN ONE DAY 



But what happens if we would like to make a forecast of stock prices further than one day ahead?

In the real world we woudn't have the indicators for make the prediction of Close price at a date further away than a day. Because we wouldn't know the previous Close prices, so we couldn't calculate the indicators that our model uses to make predictions. Unless we used indicators which were calculated from prices predicted before, and not from true prices.

We will see in the next model if we can predict Close prices for dates further than a day, based in previus predicted prices. And we will calculate the bias and variability of this new model through MAE and variance.


In [78]:
# we create here a new dataframe, copy of our clean previous dataframe, we will use it later
pred_df = df_sort.copy()

pred_df["pred_Close"] = df_sort["Close"]
pred_df["pred_avg_5"] = df_sort["avg_5"]
pred_df["pred_avg_30"] = df_sort["avg_30"]
pred_df["pred_avg_365"] = df_sort["avg_365"]

# we drop columns we don't need
pred_df.drop(pred_df.columns[1:4], axis = 1, inplace = True)
pred_df.drop(pred_df.columns[2:11], axis = 1, inplace = True)

pred_df.head()

,Date,Close,pred_Close,pred_avg_5,pred_avg_30,pred_avg_365
365,1951-06-19,22.020000,22.020000,21.800,21.703333,19.447726
366,1951-06-20,21.910000,21.910000,21.900,21.683000,19.462411
367,1951-06-21,21.780001,21.780001,21.972,21.659667,19.476274
368,1951-06-22,21.549999,21.549999,21.960,21.631000,19.489562
369,1951-06-25,21.290001,21.290001,21.862,21.599000,19.502082


In [79]:
def predictions_from_past(present_date, future_date, df, cols):
    
    # OJO dates must be present in df
    index_present = df[df["Date"] == present_date].index[0]    
    index_future = df[df["Date"] == future_date].index[0]
    
    lr = LinearRegression()
         
    X = df.loc[:index_present] # the model only makes predictions based in dates from the past
    Y = df.loc[index_future] # the observation in the future to predict
        
    lr.fit(X[cols], X["Close"])
    prediction = lr.predict([Y[cols]]) # OJO it must be in brakets, need to be array 2D
        
    return prediction[0]
    

In [84]:
# df shoud be prepared before running this function
# there should be a pred_Close column and indicators column prepared previously as copy of true data
def future_dates_predictions(date, df, cols):
    
    # date set selection from date to the end of df
    future = df[df["Date"] >= date]
    

    # iterates over all rows in future and get its index and the proper row as a serie
    for index, row in future.iterrows():

        # select former 5 values respect this row from previous predicted values, not true values
        # although in the begginig, while there aren't enough predicted values, true values are used
        formers_5 = df.loc[index - 5 : index - 1, "pred_Close"] 
        
        # calculate first indicator
        avg_5 = np.mean(formers_5)

        # asign indicator to this row in its column at df
        df.loc[index, "pred_avg_5"] = avg_5 
    
        formers_30 = df.loc[index - 30 : index - 1, "pred_Close"] 
        avg_30 = np.mean(formers_30)
        df.loc[index, "pred_avg_30"] = avg_30


        formers_365 = df.loc[index - 365 : index - 1, "pred_Close"]
        avg_365 = np.mean(formers_365)
        df.loc[index, "pred_avg_365"] = avg_365
        
        
        # make predictions based on Close price of previus observation but usin indicators calculated
        # from previous predictions
        df.loc[index, "pred_Close"] = predictions_from_past(date, df.loc[index, "Date"], df, cols)
        
        
    # returns the dataframe which stores future indicators and prices from the selected date
    return df[df["Date"] >= date]

    
# we will use the predicted cols as cols for our model in predictions_from_past function. 
# their values for dates in the future will be calculated in each iteration in future_dates_predictions
pred_cols = ["pred_avg_5", "pred_avg_30", "pred_avg_365"]

future_df = future_dates_predictions(datetime(year = 2013, month = 1, day =2), pred_df, pred_cols)

future_df.head()

,Date,Close,pred_Close,pred_avg_5,pred_avg_30,pred_avg_365
15851,2013-01-02,1462.420044,1419.209222,1418.641992,1414.258667,1327.534055
15852,2013-01-03,1459.369995,1417.448520,1417.151830,1416.236308,1327.789861
15853,2013-01-04,1466.469971,1416.855658,1416.675543,1417.254924,1327.991638
15854,2013-01-07,1461.890015,1416.511961,1416.426679,1418.223111,1328.188448
15855,2013-01-08,1457.150024,1419.482897,1419.243061,1419.072509,1328.405111


In [89]:
mae = mean_absolute_error(future_df["pred_Close"], future_df["Close"])
var = np.var(future_df["pred_Close"])
var2 = np.var(future_df["Close"])

print("MAE : " + str(round(mae, 2)))
print("Variance of predicted values : " + str(round(var, 2)))
print("Variance of true values : " + str(round(var2, 2)))


MAE : 426.34
Variance of predicted values : 340.62
Variance of true values : 36963.46



According to these error metrics, we can see our MAE has been multipled by 27 times. It seems logical due to our model now is based in predicted Close prices and not true Close prices. And our model for prediction only takes indicators of historic data, there isn't any variable that gives present information. That is why the variance of our predicted prices is too much lower than those of true prices too.

These results make sense, and show that as further are the observations we want predict, more bias our model will give us . Because the indicators used by our model, will have less and less true values. And there isn't any other variable which gives our model new information.
